In [7]:
import os
import yaml
import copy

In [8]:
# read in existing config
with open('reference_config.yaml', 'r') as f:
    config = yaml.unsafe_load(f)
print(config)

{'random-filter-s': {'parlayivf': {'docker-tag': 'neurips23-filter-parlayivf', 'module': 'neurips23.filter.parlayivf.parlayivf', 'constructor': 'ParlayIVF', 'base-args': ['@metric'], 'run-groups': {'base': {'args': '[{"cluster_size": 5000, \n  "T": 8,\n  "cutoff": 10000,\n  "max_iter": 10,\n  "weight_classes": [100000, 400000],\n  "build_params": [{"max_degree": 8,\n                    "limit": 200,\n                    "alpha": 1.175},\n                  {"max_degree": 10,\n                   "limit": 200,\n                   "alpha": 1.175},\n                  {"max_degree": 12,\n                   "limit": 200,\n                   "alpha": 1.175}],\n  "bitvector_cutoff": 10000,\n  "materialize_joins": "True"\n}]\n', 'query-args': '[{"target_points": 15000,\n  "tiny_cutoff": 60000,\n  "beam_widths": [70, 70, 70]\n  }]\n'}}}}, 'yfcc-10M': {'parlayivf': {'docker-tag': 'neurips23-filter-parlayivf', 'module': 'neurips23.filter.parlayivf.parlayivf', 'constructor': 'ParlayIVF', 'base-args'

In [9]:
# for each dataset, create a new config for each ablation
for dataset, args in config.items():
    base = args['parlayivf']
    base = copy.deepcopy(base)

    base['run-groups']['base']['args'] = yaml.load(base['run-groups']['base']['args'], Loader=yaml.FullLoader)

    # for each ablation, create a new config

    # no material join
    no_material_join = copy.deepcopy(base)
    # print(no_material_join['run-groups']['base']['args'])
    # print(type(no_material_join['run-groups']['base']['args']))
    no_material_join['run-groups']['base']['args'][0]['materialize_joins'] = 'False'
    args['parlayivf-no-material-join'] = no_material_join

    # no sorted queries
    no_sorted_queries = copy.deepcopy(base)
    no_sorted_queries['run-groups']['base']['args'][0]['sorted_queries'] = 'False'
    args['parlayivf-no-sorted-queries'] = no_sorted_queries

    # no bitvector
    no_bitvector = copy.deepcopy(base)
    no_bitvector['run-groups']['base']['args'][0]['bitvector_cutoff'] = 1_000_000_000
    args['parlayivf-no-bitvector'] = no_bitvector

    # no weight classes
    no_weight_classes = copy.deepcopy(base)
    no_weight_classes['run-groups']['base']['args'][0]['weight_classes'] = [1, 1]
    args['parlayivf-no-weight-classes'] = no_weight_classes

In [10]:
# write out new config
with open('config.yaml', 'w') as f:
    yaml.dump(config, f)